# Image Preprocessing

## Import the required libraries

In [ ]:
import cv2
import os
from matplotlib import pyplot as plt 
import numpy as np

## Create the output directory

In [ ]:
INP_DATA_DIR = "./dataset"
OUT_DATA_DIR = "./processed_dataset"

# Create a folder to store the processed images
if not os.path.exists(OUT_DATA_DIR):
    os.mkdir(OUT_DATA_DIR)

## Process the images

In [ ]:
# Iterate through the input folder and process each image for each class 
for folder in sorted(os.listdir(INP_DATA_DIR)):
    print("Processing class: ", folder)

    # Create a folder for each class in the output directory
    if not os.path.exists(os.path.join(OUT_DATA_DIR, folder)):
        os.mkdir(os.path.join(OUT_DATA_DIR, folder))

    # Iterate through each image in the input folder
    for filename in sorted(os.listdir(os.path.join(INP_DATA_DIR, folder))):
        print("Processing image: ", filename)

        # Read the image
        image = cv2.imread(os.path.join(INP_DATA_DIR, folder, filename))

        # Convert the image to the YCrCb color space
        ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

        # Apply adaptive thresholding on the Cr channel to segment the hand region
        _, cr_thres = cv2.threshold(ycrcb[:, :, 1], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Apply morphological operations to remove noise and fill holes
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        cr_thres = cv2.morphologyEx(cr_thres, cv2.MORPH_OPEN, kernel, iterations=2)
        cr_thres = cv2.dilate(cr_thres, kernel, iterations=1)

        # Find contours in the thresholded image
        contours, _ = cv2.findContours(cr_thres.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Find the contour with the largest area (assuming it's the hand)
        contour = max(contours, key=cv2.contourArea)

        # Create a mask of the hand region
        mask = np.zeros(cr_thres.shape, dtype=np.uint8)
        cv2.drawContours(mask, [contour], -1, 255, thickness=cv2.FILLED)

        # Apply the mask to the original image
        masked_image = cv2.bitwise_and(image, image, mask=mask)

        # Find the bounding box of the hand region
        x, y, w, h = cv2.boundingRect(contour)

        # Crop the hand region from the original image
        cropped_hand = masked_image[y:y+h, x:x+w]

        # Resize the cropped hand to 224x224
        cropped_hand = cv2.resize(cropped_hand, (224, 224))

        # Save the cropped hand to the output folder
        cv2.imwrite(os.path.join(OUT_DATA_DIR, folder, filename), cropped_hand)

        # Display the cropped hand using matplotlib
        plt.imshow(cv2.cvtColor(cropped_hand, cv2.COLOR_BGR2RGB))
        plt.show()